In [9]:
import os
os.chdir('../')

In [10]:
pwd

'e:\\'

In [11]:
from src.utils.utils import read_yaml,create_dir
from src.constant.ymal_path import *
from src.logging.logger import logging
from src.exception.exception import CustomException
import sys



In [ ]:
import t